In [ ]:
import time
import requests
import pandas as pd

In [ ]:
# https://www.mapdevelopers.com/draw-circle-tool.php
# Our best friend

In [ ]:
# Replace with your actual API key
api_key = "API_Key"

# Base URLs for the API endpoints
textsearch_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

def get_place_details(place_id, api_key):
    """Get phone number and website details for a given place ID."""
    details_url = (
        f"https://maps.googleapis.com/maps/api/place/details/json?"
        f"place_id={place_id}&fields=formatted_phone_number,website&key={api_key}"
    )
    details_response = requests.get(details_url)
    
    if details_response.status_code == 200:
        details = details_response.json().get('result', {})
        phone_number = details.get('formatted_phone_number', 'Not Available')
        website = details.get('website', 'Not Available')
    else:
        phone_number = 'Not Available'
        website = 'Not Available'
    
    return phone_number, website

def get_places(url, params, api_key):
    """
    Call the Places API with the provided parameters, get the results and
    return a tuple: (list_of_places, next_page_token)
    """
    data = []
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        results = json_data.get('results', [])
        next_page_token = json_data.get('next_page_token', None)
        
        for place in results:
            name = place.get('name')
            address = place.get('formatted_address')
            rating = place.get('rating', 'Not Available')
            place_id = place.get('place_id')
            
            # Get additional details
            phone_number, website = get_place_details(place_id, api_key)
            
            # Optionally, include the original search parameters in each row.
            # These keys will be added later when iterating over multiple search queries.
            row = {
                "Name": name,
                "Address": address,
                "Rating": rating,
                "Phone Number": phone_number,
                "Website": website
            }
            data.append(row)
        
        return data, next_page_token
    else:
        print("Failed to retrieve data! Status code:", response.status_code)
        return [], None


In [ ]:
key_word = "Chinese"
radius = 5280.49

# List of latitude and longitude coordinates
locations = [
    (43.7745614, -79.2282941),
    (43.8647252, -79.2722412),
    (43.7993505, -79.3106892),
    (43.710072, -79.2969575),
    (43.8766038, -79.3834787),
    (43.7973743, -79.4068092),
    (43.723953, -79.3779764),
    (43.6683648, -79.3848537),
    (43.8636894, -79.4878588),
    (43.7834851, -79.5153123),
    (43.723961, -79.4878736),
    (43.6455087, -79.4864778),
    (43.8300266, -79.6059843),
    (43.7388147, -79.6101162),
    (43.6693457, -79.582611),
    (43.5888423, -79.5702434),
    (43.7279185, -79.7377917),
    (43.675313, -79.6924665),
    (43.6067379, -79.6677524),
    (43.6713156, -79.8160581),
    (43.6117034, -79.7693687),
    (43.5365979, -79.7473974),
    (43.5216567, -79.6540198),
    (43.4524173, -79.7075727),
    (43.3773917, -79.7858503),
]

# Generate search parameters list
search_params_list = [
    {"query": key_word, "location": f"{lat},{lon}", "radius": radius}
    for lat, lon in locations
]

# List to hold all results from all searches
all_results = []

# Loop through each set of search parameters
for search_params in search_params_list:
    print(f"Searching for '{search_params['query']}' in {search_params['location']} ...")
    
    # Copy the search_params to avoid modifying the original dictionary.
    params = search_params.copy()
    params["key"] = api_key  # Add API key to the parameters
    
    # Get the first page of results
    results, next_page_token = get_places(textsearch_url, params, api_key)
    
    # If desired, include the original search criteria in each result row.
    for row in results:
        row["Query"] = search_params.get("query")
        row["Location"] = search_params.get("location")
        row["Radius"] = search_params.get("radius")
    all_results.extend(results)
    
    # Handle pagination: the API may return up to 3 pages.
    while next_page_token:
        print("Waiting for next page...")
        time.sleep(2)  # Delay before using the next_page_token (per Google's guidelines)
        params['pagetoken'] = next_page_token
        
        # Get the next page of results
        results, next_page_token = get_places(textsearch_url, params, api_key)
        for row in results:
            row["Query"] = search_params.get("query")
            row["Location"] = search_params.get("location")
            row["Radius"] = search_params.get("radius")
        all_results.extend(results)

# Convert the results to a pandas DataFrame
df = pd.DataFrame(all_results)

# Specify the column order for the Excel output
cols = ["Name", "Address", "Rating", "Phone Number", "Website"]
df = df[cols]

In [ ]:
df = df.drop_duplicates()

removed = df.shape[0] - df.drop_duplicates().shape[0]
print(f"Duplicates removed: {removed}")

In [ ]:
output_filename = "Restaurants.xlsx"
df.to_excel(output_filename, index=False)

print(f"Data saved to {output_filename}")